In [9]:
import os
from nd2reader import ND2Reader
import napari
import numpy as np
from cellpose import models
from skimage.measure import regionprops_table
from skimage.io import imsave
from skimage.segmentation import clear_border
from skimage.transform import resize
from skimage.morphology import remove_small_objects
import pandas as pd
from skimage.feature import blob_dog, blob_log
from utils import sharpest_frame_laplacian, mask_from_df
from scipy.spatial.distance import cdist

In [14]:
model = models.Cellpose(gpu=True, model_type='cyto2')

# define distances for the analysis of pairs
dist_list = [2,4,6]

# define sensitivity for the foci detection 
foci1_threshold = 0.01
foci2_threshold = 0.005

# define nulcei sizes
min_area = 2000
max_area = 20000
cell_diameter = 60

In [6]:
# pathway to the directory with the images
data_dir = r'I:\CBI\Kasia\data_analysis\2025_Fouquerel\data\53BP1 IF cenpB FISH -Lily'

# pahtway to a directory to store the results (doesn't have to exist)
masks_dir = r'I:\CBI\Kasia\data_analysis\2025_Fouquerel\analysis\250320_nuclei_masks'

os.makedirs(masks_dir, exist_ok=True)

In [11]:
# get all the nd2 files from the selected directory
file_list = [x for x in os.listdir(data_dir) if x.endswith('.nd2')]
file_list.sort()
file_list

['Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_002.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_003.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_004.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_005.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_006.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_007.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_008.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_009.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_010.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_011.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_012.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_013.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_24h_001.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_24h_002.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_24h_003.nd2',
 'Batch Deconvolution_000_clone14_siPRKDC_20minDL_24

In [12]:
# use to run the analysis only on a selected file(s)
file_list = ['Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_002.nd2']

In [ ]:
for file_name in file_list:

    im_path = os.path.join(data_dir, file_name)
    t = ND2Reader(im_path)

    # find the sharpest frame in the stack
    foci1_stack = np.array([t.get_frame_2D(c=1, z=z) for z in range(t.sizes['z'])])
    sharpest_index = sharpest_frame_laplacian(foci1_stack)

    # get images from the sharpest frame
    dapi_im = t.get_frame_2D(c=0, z=sharpest_index)
    foci1_im = t.get_frame_2D(c=1, z=sharpest_index)
    foci2_im = t.get_frame_2D(c=2, z=sharpest_index)

    # segment the image
    mask,_,_,_ = model.eval(dapi_im, diameter=cell_diameter, flow_threshold=None)

    # clear the border
    mask_border_clear = clear_border(mask)

    # save the mask
    mask_path = os.path.join(masks_dir, file_name[:-4] + '_mask.png')
    imsave(mask_path, mask_border_clear.astype(np.uint16))

    # detect foci
    foci1 = blob_log(foci1_im, min_sigma = 2, max_sigma=30, num_sigma = 10, threshold=foci1_threshold)
    foci2 = blob_log(foci2_im, min_sigma = 2, max_sigma=30, num_sigma = 10, threshold=foci2_threshold)

    # quantify cells
    df_cell = pd.DataFrame(regionprops_table(mask_border_clear, properties=('label', 'area', 'centroid','bbox','image')))
    df_cell['image'] = file_name
    df_cell['condition'] = '_'.join(file_name.split('_')[3:-1])
    df_cell['frame'] = sharpest_index

    # select cells within the size range
    df_cell = df_cell[(df_cell['area'] > min_area) & (df_cell['area'] < max_area)]
    df_cell = df_cell.reset_index(drop=True)

    # recreate nuclei mask
    mask_border_clear = mask_from_df(df_cell, shape = mask_border_clear.shape)
    # drop the image column
    df_cell = df_cell.drop(columns=['image'])
    df_cell.columns = ['cell', 'area', 'centroid-0', 'centroid-1', 'bbox-0', 'bbox-1', 'bbox-2', 'bbox-3']

    # quantify foci
    df_foci1 = pd.DataFrame(foci1, columns=['y', 'x', 'r'])
    df_foci1['cell'] = mask_border_clear[foci1[:,0].astype(int), foci1[:,1].astype(int)]

    df_foci2 = pd.DataFrame(foci2, columns=['y', 'x', 'r'])
    df_foci2['cell'] = mask_border_clear[foci2[:,0].astype(int), foci2[:,1].astype(int)]

    # count foci per cell
    df_foci1_count = df_foci1.groupby('cell').size().reset_index(name='foci1_count')
    df_foci2_count = df_foci2.groupby('cell').size().reset_index(name='foci2_count')
    df_cell = df_cell.merge(df_foci1_count, on='cell', how='left').merge(df_foci2_count, on='cell', how='left')

    # quantify foci pairs
    for cell in df_cell['cell']:
        
        # Select points for this cell
        foci1_cell = df_foci1.loc[df_foci1['cell'] == cell,['y', 'x']].values
        foci2_cell = df_foci2.loc[df_foci2['cell'] == cell,['y', 'x']].values
        
        if len(foci1_cell) == 0 or len(foci2_cell) == 0:
            continue

        # Compute pairwise distances
        distances = cdist(foci1_cell, foci2_cell, metric='euclidean')
        
        for d in dist_list:
            count = np.sum(distances < d)
            df_cell.loc[df_cell['cell'] == cell, f'foci1_foci2_{d}'] = count

    # save the data
    df_cell.to_csv(os.path.join(masks_dir, file_name[:-4] + '_cell_data.csv'), index=False)
    df_foci1.to_csv(os.path.join(masks_dir, file_name[:-4] + '_foci1_data.csv'), index=False)
    df_foci2.to_csv(os.path.join(masks_dir, file_name[:-4] + '_foci2_data.csv'), index=False)

c:\Users\KMK280\AppData\Local\miniconda3\envs\cellpose-env\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: I:\CBI\Kasia\data_analysis\2025_Fouquerel\analysis\250320_nuclei_masks\Batch Deconvolution_000_clone14_siPRKDC_20minDL_0h_002_mask.png is a low contrast image
  return func(*args, **kwargs)


KeyError: 'label'